In [2]:
!pip install tqdm
!pip install torch
!pip install torchvision torchaudio
!pip install tensorboardX
!pip install scikit-learn
!pip install pytorch-lightning

In [1]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from dataset import MRDataset, MRDatasetMerged
from torch.utils.data import DataLoader 
import model

import pytorch_lightning as pl
from sklearn import metrics


In [2]:
pl.__version__

'1.3.1'

In [3]:
import multiprocessing

num_workers = multiprocessing.cpu_count() - 1
num_workers

11

In [4]:
class Args:
    def __init__(self):
        self.task = "abnormal" #['abnormal', 'acl', 'meniscus']
        self.plane = "sagittal" #['sagittal', 'coronal', 'axial']
        self.prefix_name = "Test"
        self.augment = 1 #[0, 1]
        self.lr_scheduler = "plateau" #['plateau', 'step']
        self.gamma = 0.5
        self.epochs = 1
        self.lr = 1e-5
        self.flush_history = 0 #[0, 1]
        self.save_model = 1 #[0, 1]
        self.patience = 5
        self.log_every = 100
        
args = Args()

In [5]:
def to_tensor(x):
    return torch.Tensor(x)

In [ ]:
log_root_folder = "./logs/{0}/{1}/".format(args.task, args.plane)
if args.flush_history == 1:
    objects = os.listdir(log_root_folder)
    for f in objects:
        if os.path.isdir(log_root_folder + f):
            shutil.rmtree(log_root_folder + f)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)

# augmentor = Compose([
#     transforms.Lambda(to_tensor),
#     RandomRotate(25),
#     RandomTranslate([0.11, 0.11]),
#     RandomFlip(),
# #     transforms.Lambda(lambda x: x.repeat(3, 1, 1, 1).permute(1, 0, 2, 3)),
# ])

train_dataset = MRDatasetMerged('./data/', transform=None, train=True)
validation_dataset = MRDatasetMerged('./data/', train=False)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=num_workers, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, num_workers=num_workers, drop_last=False)

mrnet = model.MRNet()
trainer = pl.Trainer(max_epochs=10)

trainer.fit(mrnet, train_loader, validation_loader)

/Users/rob/anaconda3/envs/mrnet/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type              | Params
-------------------------------------------------------
0 | pretrained_model | AlexNet           | 61.1 M
1 | pooling_layer    | AdaptiveAvgPool2d | 0     
2 | classifer        | Linear            | 771   
3 | train_f1         | F1                | 0     
4 | valid_f1         | F1                | 0     
5 | train_auc        | AUROC             | 0     
-------------------------------------------------------
61.1 M    Trainable params
0         Non-trainable params
61.1 M    Total params
244.406   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/rob/anaconda3/envs/mrnet/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Training: 0it [00:00, ?it/s]

In [7]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

- Train model for all planes & diagnosis --> One Net to rule them all
    - Binary --> multilabel
    - one plane per dataset --> planes merged for series
- Inference pipeline: Load model, predict on all planes for single sample --> return proba

Features for performance boost
- Add metric (rocauc)
- Augmentations: readd (albumentations) (minimal pipeline ++)
- alexnet to resnet
- XGBoost for boosting of classifiers 
